In [1]:
from Moduled_functions import get_data
from Moduled_functions import tranformation
from Moduled_functions import calculate_ema_slope
from Moduled_functions import place_order
from m_email import send_email
import pandas as pd
from keras.models import load_model
from sklearn.preprocessing import StandardScaler

2025-04-06 11:58:35.213299: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-06 11:58:35.216803: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-06 11:58:35.247605: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-06 11:58:35.247663: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-06 11:58:35.248541: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
import os

# Create directory structure
def create_directory_structure():
    directories = [
        'data',
        'data/raw',
        'data/processed',
        'data/signals',
        'data/orders',
        'data/predictions',
        'logs',
        'config'
    ]
    
    for directory in directories:
        os.makedirs(directory, exist_ok=True)
        print(f"Created directory: {directory}")

create_directory_structure()

Created directory: data
Created directory: data/raw
Created directory: data/processed
Created directory: data/signals
Created directory: data/orders
Created directory: data/predictions
Created directory: models
Created directory: logs
Created directory: config


In [7]:
# Combined trading system framework
import pandas as pd
import numpy as np
from datetime import datetime
import requests
from abc import ABC, abstractmethod
import logging
import json
from Moduled_functions import get_data

# Configure logging
logging.basicConfig(
    filename='logs/trading_system.log',
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger('TradingSystem')
class BaseStrategy(ABC):
    def __init__(self, name):
        self.name = name
        self.signals_path = f'data/signals/signals_{name}.csv'
        self.trades_path = f'data/orders/trades_{name}.csv'
        self.config_path = f'config/strategy_{name}.json'
        self.load_config()
    
    def load_config(self):
        """Load strategy configuration"""
        try:
            with open(self.config_path, 'r') as f:
                self.config = json.load(f)
        except FileNotFoundError:
            self.config = self.get_default_config()
            self.save_config()
    
    def save_config(self):
        """Save strategy configuration"""
        with open(self.config_path, 'w') as f:
            json.dump(self.config, f, indent=4)
    
    @abstractmethod
    def get_default_config(self):
        """Get default strategy configuration"""
        pass
    
    @abstractmethod
    def generate_signals(self, data):
        """Generate trading signals based on strategy logic"""
        pass
    
    @abstractmethod
    def get_order_parameters(self, current_price, signal):
        """Get order parameters based on strategy signal"""
        pass
    
    def save_signals(self, signals_data):
        """Save strategy signals to history"""
        try:
            if os.path.exists(self.signals_path):
                existing_signals = pd.read_csv(self.signals_path)
                signals_data = signals_data.iloc[-1:]
                updated_signals = pd.concat([existing_signals, signals_data])
            else:
                updated_signals = signals_data
            updated_signals.to_csv(self.signals_path, index=False)
            logger.info(f"Signals saved for strategy {self.name}")
        except Exception as e:
            logger.error(f"Error saving signals for strategy {self.name}: {str(e)}")
    
    def save_trade(self, trade_data):
        """Save trade details to history"""
        try:
            if os.path.exists(self.trades_path):
                existing_trades = pd.read_csv(self.trades_path)
                updated_trades = pd.concat([existing_trades, trade_data])
            else:
                updated_trades = trade_data
            updated_trades.to_csv(self.trades_path, index=False)
            logger.info(f"Trade saved for strategy {self.name}")
        except Exception as e:
            logger.error(f"Error saving trade for strategy {self.name}: {str(e)}")

class StrategyRegistry:
    _strategies = {}
    
    @classmethod
    def register(cls, strategy_instance):
        """Register a new strategy"""
        cls._strategies[strategy_instance.name] = strategy_instance
        logger.info(f"Strategy registered: {strategy_instance.name}")
    
    @classmethod
    def get_strategy(cls, name):
        """Get a registered strategy by name"""
        return cls._strategies.get(name)
    
    @classmethod
    def get_all_strategies(cls):
        """Get all registered strategies"""
        return cls._strategies

In [89]:
class DataHandler:
    def __init__(self):
        self.scaler = None
        self.raw_data_path = 'data/raw/base_data.csv'
        self.processed_data_path = 'data/processed/transformed_data.csv'
    
    def get_and_prepare_data(self, lookback, access_token):
        from Moduled_functions import tranformation
        """Get and prepare the base data"""
        try:
            base_data, current_price = get_data(lookback, access_token)
            #base_data_append=base_data.iloc[-1:]

        # Read existing history
            try:
                existing_data = pd.read_csv(self.raw_data_path,index=False)
                existing_data['datetime'] = pd.to_datetime(existing_data['datetime'])
                base_data['datetime'] = pd.to_datetime(base_data['datetime'])
                
                
                # Add new records
                new_records = base_data[~base_data['datetime'].isin(existing_data['datetime'])]
                if not new_records.empty:
                    updated_data = pd.concat([existing_data, new_records])
                    updated_data = updated_data.sort_values('datetime')
                else:
                    updated_data = existing_data
            except FileNotFoundError:
                updated_data = base_data

            updated_data.to_csv(self.raw_data_path, index=False)
            logger.info(f"Raw data saved to {self.raw_data_path}")
            
            base_data = tranformation(base_data, 0.0015)            
            return base_data, current_price
        except Exception as e:
            logger.error(f"Error in get_and_prepare_data: {str(e)}")
            raise
    
    def calculate_technical_indicators(self, data):
        """Calculate technical indicators"""
        try:
            data['EMA_slope'] = calculate_ema_slope(data, 'Open', 9)
            data['EMA_slope_15'] = calculate_ema_slope(data, 'Open', 15)
            data['EMA_slope_60'] = calculate_ema_slope(data, 'Open', 60)
            data['deviation_'] = data['bullish_move_flag_20'] + data['bearish_move_flag_20']
            
            try:
                # Read existing history
                existing_data = pd.read_csv(self.processed_data_path,index=False)
                existing_data['datetime'] = pd.to_datetime(existing_data['datetime'])
                data['datetime'] = pd.to_datetime(data['datetime'])
                # Add new records
                new_records = data[~data['datetime'].isin(existing_data['datetime'])]
                if not new_records.empty:
                    updated_data = pd.concat([existing_data, new_records])
                    updated_data = updated_data.sort_values('datetime')
                else:
                    updated_data = existing_data
            except FileNotFoundError:
                updated_data = data 

            updated_data.to_csv(self.processed_data_path, index=False)

            logger.info(f"Processed data saved to {self.processed_data_path}")
            return data
        except Exception as e:
            logger.error(f"Error in calculate_technical_indicators: {str(e)}")
            raise
    
    def normalize_features(self, data):
        """Normalize the feature data"""
        try:
            X = data.iloc[:,9:69]
            y = data['deviation_']
            
            if self.scaler is None:
                self.scaler = StandardScaler()
                self.scaler.fit(X)
            
            X_normalized = pd.DataFrame(self.scaler.transform(X)).round(1)
            return X_normalized, y
        except Exception as e:
            logger.error(f"Error in normalize_features: {str(e)}")
            raise
class OrderManager:
    def __init__(self, access_token):
        self.access_token = access_token
        #self.account_id = account_id
        self.orders_path = 'data/orders/all_orders.csv'
    
    def place_order(self, current_price, stop_loss, take_profit, quantity,account_id):
        """Place an order"""
        try:
            order = place_order(
                str(current_price),
                str(round(stop_loss, 5)),
                str(round(take_profit, 5)),
                quantity,
                self.access_token,
                account_id
            )
            logger.info(f"Order placed: {order['orderCreateTransaction'].get('id')}")
            return order
        except Exception as e:
            logger.error(f"Error placing order: {str(e)}")
            raise
    
    def save_order(self, order_data):
        """Save order details to history"""
        try:
            if os.path.exists(self.orders_path):
                existing_orders = pd.read_csv(self.orders_path)
                updated_orders = pd.concat([existing_orders, order_data])
            else:
                updated_orders = order_data
            updated_orders.to_csv(self.orders_path, index=False)
            logger.info("Order saved to history")
        except Exception as e:
            logger.error(f"Error saving order: {str(e)}")

In [87]:
class TradingSystem:
    def __init__(self, access_token, bearish_model, bullish_model):
        self.data_handler = DataHandler()
        self.order_manager = OrderManager(access_token)
        self.predictions_data_path = 'data/predictions/predictions_data.csv'
        #self.bearish_model = bearish_model
        #self.bullish_model = bullish_model
        
    def execute(self, lookback=100):
        try:
            # Get and prepare data
            base_data, current_price = self.data_handler.get_and_prepare_data(lookback, self.order_manager.access_token)
            base_data = self.data_handler.calculate_technical_indicators(base_data)
            X_normalized, y = self.data_handler.normalize_features(base_data)
            
            # Generate predictions
            
            y_pred_bearish = bearish_model.predict(X_normalized, verbose=0)
            y_pred_bearish = pd.DataFrame(y_pred_bearish)
            y_pred_bullish = bullish_model.predict(X_normalized, verbose=0)
            y_pred_bullish = pd.DataFrame(y_pred_bullish)
            
            # Combine results
            result_combined = pd.concat([y_pred_bearish, y_pred_bullish, y], axis=1)
            result_combined.columns = ['bearish_0','bearish_1','bearish_2','bearish_3',
                                    'bullish_0','bullish_1','bullish_2','bullish_3','y_true']
            
            result_combined = pd.concat([base_data['datetime'], result_combined], axis=1)

            try:
                # Read existing history
                existing_data = pd.read_csv(self.predictions_data_path,index=False)
                existing_data['datetime'] = pd.to_datetime(existing_data['datetime'])
                result_combined['datetime'] = pd.to_datetime(result_combined['datetime'])
                
                
                # Add new records
                new_records = result_combined[~result_combined['datetime'].isin(existing_data['datetime'])]
                if not new_records.empty:
                    updated_data = pd.concat([existing_data, new_records])
                    updated_data = updated_data.sort_values('datetime')
                else:
                    updated_data = existing_data
            except FileNotFoundError:
                updated_data = result_combined
                
            updated_data.to_csv(self.predictions_data_path, index=False)

            del(result_combined)
            # Process each strategy
            for strategy in StrategyRegistry.get_all_strategies().values():
                self._process_strategy(strategy, base_data, current_price)
            
            logger.info(f"Execution completed at {datetime.now()}")
            
        except Exception as e:
            logger.error(f"Error in execute: {str(e)}")
    
    def _process_strategy(self, strategy, base_data, current_price):
        """Process individual strategy signals and execute trades"""
        try:
            # Generate signals
            signals = strategy.generate_signals(base_data)
            signals['timestamp'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            strategy.save_signals(signals)
            
            # Check signals and execute trades
            latest_bearish = signals['bearish_flag'].iloc[-1]
            latest_bullish = signals['bullish_flag'].iloc[-1]
            
            if latest_bullish == 1 and latest_bearish != 2:
                self._execute_trade(strategy, 1, current_price)
            elif latest_bearish == 2 and latest_bullish != 1:
                self._execute_trade(strategy, 2, current_price)
                
        except Exception as e:
            logger.error(f"Error processing strategy {strategy.name}: {str(e)}")
    
    def _execute_trade(self, strategy, signal, current_price):
        """Execute and record a trade for a strategy"""
        try:
            order_params = strategy.get_order_parameters(current_price, signal)
            if not order_params:
                return
            
            order = self.order_manager.place_order(
                current_price,
                order_params['stop_loss'],
                order_params['take_profit'],
                order_params['quantity'],
                order_params['account_id']
            )
            
            order_id = order['orderCreateTransaction'].get('id')
            order_time = order['orderCreateTransaction'].get('time')
            account_id = order['orderCreateTransaction'].get('accountID')
            
            # Save trade details
            trade_data = pd.DataFrame({
                'Time': [order_time],
                'Strategy': [strategy.name],
                'Type': [order_params['type']],
                'Quantity': [order_params['quantity']],
                'Price': [current_price],
                'Take_Profit': [order_params['take_profit']],
                'Stop_Loss': [order_params['stop_loss']],
                'Order_ID': [order_id],
                'Account_ID': [account_id],
            })
            
            strategy.save_trade(trade_data)
            self.order_manager.save_order(trade_data)
            
            # Send notification
            message = f"{strategy.name} {order_params['type']} Order placed with order no: {order_id}"
            requests.get(f'https://api.day.app/iFbt9PqBdm6d2YvUT4irnN/{order_params["type"]} Order Placed/{message}')
            logger.info(message)
            
        except Exception as e:
            logger.error(f"Error executing trade for strategy {strategy.name}: {str(e)}")

print("Trading system framework initialized with directory structure")

Trading system framework initialized with directory structure


In [91]:
# Example implementations of concrete strategies
class VolumeBasedStrategy(BaseStrategy):
    def __init__(self):
        super().__init__('volume_based')
    
    def get_default_config(self):
        return {
            'ema_slope_threshold': 450,
            'ema_slope_15_threshold': 750,
            'ema_slope_60_threshold': 0.50,
            'bearish_threshold_max': 0.65,
            'move_from_top_min': 0.08,
            'move_from_top_max': 0.25,
            'volume_min': 450,
            'volume_max': 700,
            'volume_mean_3_min': 500,
            'volume_mean_3_max': 750,
            'take_profit_multiplier': 1.0015,
            'stop_loss_multiplier': 0.0005,
            'quantity': '1000000'
        }
    
    def generate_signals(self, data):
        config = self.config
        
        # Generate bearish signals
        bearish_signals = data.apply(lambda x: 2 if (
            x['EMA_slope'] < config['ema_slope_threshold'] and
            x['EMA_slope_15'] < config['ema_slope_15_threshold'] and
            x['EMA_slope_60'] < config['ema_slope_60_threshold'] and
            x['volume'] > config['volume_threshold']
        ) else 0, axis=1)
        
        # Generate bullish signals
        bullish_signals = data.apply(lambda x: 1 if (
            x['EMA_slope'] > abs(config['ema_slope_threshold']) and
            x['EMA_slope_15'] > abs(config['ema_slope_15_threshold']) and
            x['EMA_slope_60'] > abs(config['ema_slope_60_threshold']) and
            x['volume'] > config['volume_threshold']
        ) else 0, axis=1)
        
        signals = pd.DataFrame({
            'datetime': data['datetime'],
            'bearish_flag': bearish_signals,
            'bullish_flag': bullish_signals
        })
        
        return signals
    
    def get_order_parameters(self, current_price, signal):
        config = self.config
        
        if signal == 1:  # Bullish
            return {
                'take_profit': current_price * config['take_profit_multiplier'],
                'stop_loss': current_price * config['stop_loss_multiplier'],
                'quantity': config['quantity'],
                'type': 'BUY',
                'account_id' : '101-004-31059296-001'
            }
        elif signal == 2:  # Bearish
            return {
                'take_profit': current_price * (2 - config['take_profit_multiplier']),
                'stop_loss': current_price * config['stop_loss_multiplier'],
                'quantity': f"-{config['quantity']}",
                'type': 'SELL',
                'account_id' : '101-004-31059296-001'
            }
        return None

# Example implementations of concrete strategies
class VolumeBasedStrategyHedge(BaseStrategy):
    def __init__(self):
        super().__init__('volume_based_hedge')
    
    def get_default_config(self):
        return {
            'ema_slope_threshold': 450,
            'ema_slope_15_threshold': 750,
            'ema_slope_60_threshold': 0.50,
            'bearish_threshold_max': 0.65,
            'move_from_top_min': 0.08,
            'move_from_top_max': 0.25,
            'volume_min': 450,
            'volume_max': 700,
            'volume_mean_3_min': 500,
            'volume_mean_3_max': 750,
            'take_profit_multiplier': 1.0015,
            'stop_loss_multiplier': 0.0005,
            'quantity': '1000000'
        }
    
    def generate_signals(self, data):
        config = self.config
        
        # Generate bearish signals
        bearish_signals = data.apply(lambda x: 2 if (
            x['EMA_slope'] < config['ema_slope_threshold'] and
            x['EMA_slope_15'] < config['ema_slope_15_threshold'] and
            x['EMA_slope_60'] < config['ema_slope_60_threshold'] and
            x['volume'] > config['volume_threshold']
        ) else 0, axis=1)
        
        # Generate bullish signals
        bullish_signals = data.apply(lambda x: 1 if (
            x['EMA_slope'] > abs(config['ema_slope_threshold']) and
            x['EMA_slope_15'] > abs(config['ema_slope_15_threshold']) and
            x['EMA_slope_60'] > abs(config['ema_slope_60_threshold']) and
            x['volume'] > config['volume_threshold']
        ) else 0, axis=1)
        
        signals = pd.DataFrame({
            'datetime': data['datetime'],
            'bearish_flag': bearish_signals,
            'bullish_flag': bullish_signals
        })
        
        return signals
    
    def get_order_parameters(self, current_price, signal):
        config = self.config
        
        if signal == 2:  # Bullish
            return {
                'take_profit': current_price * config['take_profit_multiplier'],
                'stop_loss': current_price * config['stop_loss_multiplier'],
                'quantity': config['quantity'],
                'type': 'BUY',
                'account_id' : '101-004-31059296-003'
            }
        elif signal == 1:  # Bearish
            return {
                'take_profit': current_price * (2 - config['take_profit_multiplier']),
                'stop_loss': current_price * config['stop_loss_multiplier'],
                'quantity': f"-{config['quantity']}",
                'type': 'BUY',
                'account_id' : '101-004-31059296-003'
            }
        return None

class EMASlopeStrategy(BaseStrategy):
    def __init__(self):
        super().__init__('ema_slope')
    
    def get_default_config(self):
        return {
            'ema_slope_threshold': -0.0002,
            'ema_slope_15_threshold': -0.0001,
            'ema_slope_60_threshold': 0,
            'volume_threshold': 500,
            'take_profit_multiplier': 1.002,
            'stop_loss_multiplier': 0.001,
            'quantity': '500000'
        }
    
    def generate_signals(self, data):
        config = self.config
        
        # Generate bearish signals
        bearish_signals = data.apply(lambda x: 2 if (
            x['EMA_slope'] < config['ema_slope_threshold'] and
            x['EMA_slope_15'] < config['ema_slope_15_threshold'] and
            x['EMA_slope_60'] < config['ema_slope_60_threshold'] and
            x['volume'] > config['volume_threshold']
        ) else 0, axis=1)
        
        # Generate bullish signals
        bullish_signals = data.apply(lambda x: 1 if (
            x['EMA_slope'] > abs(config['ema_slope_threshold']) and
            x['EMA_slope_15'] > abs(config['ema_slope_15_threshold']) and
            x['EMA_slope_60'] > abs(config['ema_slope_60_threshold']) and
            x['volume'] > config['volume_threshold']
        ) else 0, axis=1)
        
        signals = pd.DataFrame({
            'datetime': data['datetime'],
            'bearish_flag': bearish_signals,
            'bullish_flag': bullish_signals
        })
        
        return signals
    
    def get_order_parameters(self, current_price, signal):
        config = self.config
        
        if signal == 1:  # Bullish
            return {
                'take_profit': current_price * config['take_profit_multiplier'],
                'stop_loss': current_price * config['stop_loss_multiplier'],
                'quantity': config['quantity'],
                'type': 'BUY',
                'account_id' : '101-004-31059296-001'
            }
        elif signal == 2:  # Bearish
            return {
                'take_profit': current_price * (2 - config['take_profit_multiplier']),
                'stop_loss': current_price * config['stop_loss_multiplier'],
                'quantity': f"-{config['quantity']}",
                'type': 'SELL',
                'account_id' : '101-004-31059296-001'
            }
        return None

print("Strategy implementations completed")

Strategy implementations completed


In [93]:
access_token = '320c03e29b3723c3869a2eafa278c553-c96eccee26e4e5864dcf001e98f7db84'
account_id = '101-004-31059296-001'
bearish_model = load_model('/work/Trained_Models/FX_Bearish_model_2025-02-19.keras')
bullish_model = load_model('/work/Trained_Models/FX_Bullish_model_2025-02-21.keras')


# Initialize the system
trading_system = TradingSystem(access_token, bearish_model, bullish_model)

# Register strategies
volume_strategy = VolumeBasedStrategy()
volume_strategy_hedge = VolumeBasedStrategyHedge()
ema_strategy = EMASlopeStrategy()

StrategyRegistry.register(volume_strategy)
StrategyRegistry.register(volume_strategy_hedge)
StrategyRegistry.register(ema_strategy)

# Execute trading system
trading_system.execute(lookback=100)

2025-04-06 13:29:25,161 - INFO - Strategy registered: volume_based
2025-04-06 13:29:25,163 - INFO - Strategy registered: volume_based_hedge
2025-04-06 13:29:25,163 - INFO - Strategy registered: ema_slope
2025-04-06 13:29:25,251 - ERROR - Error in get_and_prepare_data: read_csv() got an unexpected keyword argument 'index'
2025-04-06 13:29:25,253 - ERROR - Error in execute: read_csv() got an unexpected keyword argument 'index'


In [29]:
import os

# Check if logs directory exists and create if not
log_dir = 'logs'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# Test if log file is being written
import logging

# Configure logging with both file and console handlers
logger = logging.getLogger('TradingSystem')
logger.setLevel(logging.INFO)

# Clear any existing handlers
logger.handlers = []

# Create file handler
file_handler = logging.FileHandler('logs/trading_system.log')
file_handler.setLevel(logging.INFO)
file_formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
file_handler.setFormatter(file_formatter)

# Create console handler
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)
console_formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
console_handler.setFormatter(console_formatter)

# Add both handlers to logger
logger.addHandler(file_handler)
logger.addHandler(console_handler)

# Test logging
logger.info("Test log message - checking if logging is working")

# Read and display the last few lines of the log file
try:
    with open('logs/trading_system.log', 'r') as f:
        last_lines = f.readlines()[-5:]  # Get last 5 lines
        print("\nLast few lines from log file:")
        for line in last_lines:
            print(line.strip())
except FileNotFoundError:
    print("Log file not found!")
except Exception as e:
    print(f"Error reading log file: {str(e)}")

2025-04-06 12:02:26,280 - INFO - Test log message - checking if logging is working

Last few lines from log file:
2025-04-06 11:59:28,414 - TradingSystem - INFO - Signals saved for strategy volume_based_hedge
2025-04-06 11:59:28,868 - TradingSystem - INFO - Signals saved for strategy ema_slope
2025-04-06 11:59:28,868 - TradingSystem - INFO - Execution completed at 2025-04-06 11:59:28.868861
2025-04-06 12:02:26,280 - TradingSystem - INFO - Test log message - checking if logging is working
2025-04-06 12:02:26,280 - TradingSystem - INFO - Test log message - checking if logging is working


In [ ]:
print(f"Check completed at {current_time}")

In [ ]:
# Set up the schedule to run at specified intervals
for minute in [1, 6, 11, 16, 21, 26, 31, 36, 41, 46, 51, 56]:
    schedule.every().hour.at(f":{minute:02d}").do(job)

In [ ]:
print("Starting the trading bot...")
while True:
    schedule.run_pending()
    time.sleep(1)

In [83]:
link = '/work/data/raw/base_data.csv'
ss = pd.read_csv(link)

FileNotFoundError: [Errno 2] No such file or directory: '/work/data/raw/base_data.csv'

In [85]:
pd.read_csv(link)

,Unnamed: 0,datetime,Open,High,Low,Close,volume
0,0,2025-04-03 03:20:00+00:00,10914.6,10917.2,10911.6,10916.4,702
1,1,2025-04-03 03:25:00+00:00,10916.3,10920.5,10916.1,10919.7,574
2,2,2025-04-03 03:30:00+00:00,10920.0,10921.2,10917.4,10917.6,640
3,3,2025-04-03 03:35:00+00:00,10917.6,10921.1,10915.5,10915.7,469
4,4,2025-04-03 03:40:00+00:00,10915.6,10916.2,10911.6,10911.6,590
...,...,...,...,...,...,...,...
492,492,2025-04-04 20:20:00+00:00,10951.9,10951.9,10936.2,10937.6,802
493,493,2025-04-04 20:25:00+00:00,10937.4,10939.2,10932.6,10934.1,867
494,494,2025-04-04 20:30:00+00:00,10934.2,10944.6,10933.4,10944.5,696
495,495,2025-04-04 20:35:00+00:00,10944.6,10948.6,10943.8,10943.8,528


In [79]:
1ss.iloc[-1:]

,datetime,bearish_0,bearish_1,bearish_2,bearish_3,bullish_0,bullish_1,bullish_2,bullish_3,y_true
498,2025-04-04 20:50:00+00:00,0.477345,1.030867e-10,0.522655,1.030867e-10,0.487912,0.512088,1.295323e-10,1.295998e-10,0


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a127aa9e-0a77-4af9-a6ce-85e7a9b74042' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>